# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt

from torch import cuda

from absa_models.ABSA_BERT_Dropout_Linear import ABSA_BERT_Dropout_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8
SEQ_LEN = 512

NO_RUNS = 10

In [10]:
BERT_FINE_TUNED_PATH = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_512.pth'

In [11]:
MODEL_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_linear_512.pth'
STATS_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_linear_512.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    losses = []

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))

        losses.append(loss.item())
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()
    
    return losses

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_Linear(torch.load(BERT_FINE_TUNED_PATH), dropout=0.3, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    train_losses = []

    for epoch in range(EPOCHS):
        losses = train(epoch, model, loss_fn, optimizer, train_dataloader)

        train_losses += losses
    
    plt.title(f'Train Loss for run {i + 1}/{NO_RUNS}')
    plt.plot(train_losses)
    plt.savefig(f'../../../results/ABSA/SemEval16 - Task 5 - Restaurants/plots/bert_ft_do_lin/train_loss_run_{i + 1}.png')

    plt.clf()

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.502254843711853
Epoch: 0/2, Batch: 50/501, Loss: 0.016529351472854614
Epoch: 0/2, Batch: 100/501, Loss: 0.01436641626060009
Epoch: 0/2, Batch: 150/501, Loss: 0.0038785499054938555
Epoch: 0/2, Batch: 200/501, Loss: 0.013922779820859432
Epoch: 0/2, Batch: 250/501, Loss: 0.005508235655725002
Epoch: 0/2, Batch: 300/501, Loss: 0.00804922915995121
Epoch: 0/2, Batch: 350/501, Loss: 0.005616351496428251
Epoch: 0/2, Batch: 400/501, Loss: 0.006902816239744425
Epoch: 0/2, Batch: 450/501, Loss: 0.014564670622348785
Epoch: 0/2, Batch: 500/501, Loss: 0.009878995828330517
Epoch: 1/2, Batch: 0/501, Loss: 0.011151284910738468
Epoch: 1/2, Batch: 50/501, Loss: 0.006698111537843943
Epoch: 1/2, Batch: 100/501, Loss: 0.004010316915810108
Epoch: 1/2, Batch: 150/501, Loss: 0.00269548618234694
Epoch: 1/2, Batch: 200/501, Loss: 0.0037554088048636913
Epoch: 1/2, Batch: 250/501, Loss: 0.01226247102022171
Epoch: 1/2, Batch: 300/501, Loss: 0.00223394762724638
Epoch: 1/2, B

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/501, Loss: 1.463208556175232
Epoch: 0/2, Batch: 50/501, Loss: 0.0261706355959177
Epoch: 0/2, Batch: 100/501, Loss: 0.02632233500480652
Epoch: 0/2, Batch: 150/501, Loss: 0.009504584595561028
Epoch: 0/2, Batch: 200/501, Loss: 0.010320761241018772
Epoch: 0/2, Batch: 250/501, Loss: 0.013255215249955654
Epoch: 0/2, Batch: 300/501, Loss: 0.00992375984787941
Epoch: 0/2, Batch: 350/501, Loss: 0.007401903625577688
Epoch: 0/2, Batch: 400/501, Loss: 0.010724782943725586
Epoch: 0/2, Batch: 450/501, Loss: 0.0063756671734154224
Epoch: 0/2, Batch: 500/501, Loss: 0.002112202811986208
Epoch: 1/2, Batch: 0/501, Loss: 0.013976559974253178
Epoch: 1/2, Batch: 50/501, Loss: 0.008522032760083675
Epoch: 1/2, Batch: 100/501, Loss: 0.007790213916450739
Epoch: 1/2, Batch: 150/501, Loss: 0.006083075888454914
Epoch: 1/2, Batch: 200/501, Loss: 0.017984537407755852
Epoch: 1/2, Batch: 250/501, Loss: 0.0035385589580982924
Epoch: 1/2, Batch: 300/501, Loss: 0.012331734411418438
Epoch: 1/2, 

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.378906488418579
Epoch: 0/2, Batch: 50/501, Loss: 0.02422570064663887
Epoch: 0/2, Batch: 100/501, Loss: 0.022561803460121155
Epoch: 0/2, Batch: 150/501, Loss: 0.015224645845592022
Epoch: 0/2, Batch: 200/501, Loss: 0.007838728837668896
Epoch: 0/2, Batch: 250/501, Loss: 0.012695328332483768
Epoch: 0/2, Batch: 300/501, Loss: 0.005160419270396233
Epoch: 0/2, Batch: 350/501, Loss: 0.008263268508017063
Epoch: 0/2, Batch: 400/501, Loss: 0.011839320883154869
Epoch: 0/2, Batch: 450/501, Loss: 0.006531660910695791
Epoch: 0/2, Batch: 500/501, Loss: 0.007484905421733856
Epoch: 1/2, Batch: 0/501, Loss: 0.003023012075573206
Epoch: 1/2, Batch: 50/501, Loss: 0.018769459798932076
Epoch: 1/2, Batch: 100/501, Loss: 0.003990563564002514
Epoch: 1/2, Batch: 150/501, Loss: 0.006824523210525513
Epoch: 1/2, Batch: 200/501, Loss: 0.005231273826211691
Epoch: 1/2, Batch: 250/501, Loss: 0.00479878019541502
Epoch: 1/2, Batch: 300/501, Loss: 0.005909568164497614
Epoch: 1/2, 

...

Run 9/10
Epoch: 0/2, Batch: 0/501, Loss: 1.0988523960113525
Epoch: 0/2, Batch: 50/501, Loss: 0.02853679284453392
Epoch: 0/2, Batch: 100/501, Loss: 0.016795320436358452
Epoch: 0/2, Batch: 150/501, Loss: 0.012678098864853382
Epoch: 0/2, Batch: 200/501, Loss: 0.004039342049509287
Epoch: 0/2, Batch: 250/501, Loss: 0.0137419318780303
Epoch: 0/2, Batch: 300/501, Loss: 0.010943427681922913
Epoch: 0/2, Batch: 350/501, Loss: 0.0077042654156684875
Epoch: 0/2, Batch: 400/501, Loss: 0.02211759053170681
Epoch: 0/2, Batch: 450/501, Loss: 0.012345709837973118
Epoch: 0/2, Batch: 500/501, Loss: 0.007663092575967312
Epoch: 1/2, Batch: 0/501, Loss: 0.006061469670385122
Epoch: 1/2, Batch: 50/501, Loss: 0.004453578963875771
Epoch: 1/2, Batch: 100/501, Loss: 0.007163146045058966
Epoch: 1/2, Batch: 150/501, Loss: 0.005837799981236458
Epoch: 1/2, Batch: 200/501, Loss: 0.009893402457237244
Epoch: 1/2, Batch: 250/501, Loss: 0.006726175080984831
Epoch: 1/2, Batch: 300/501, Loss: 0.005032689776271582
Epoch: 1/2, 

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4918681383132935
Epoch: 0/2, Batch: 50/501, Loss: 0.023114100098609924
Epoch: 0/2, Batch: 100/501, Loss: 0.02297290228307247
Epoch: 0/2, Batch: 150/501, Loss: 0.010746589861810207
Epoch: 0/2, Batch: 200/501, Loss: 0.009593754075467587
Epoch: 0/2, Batch: 250/501, Loss: 0.0066449022851884365
Epoch: 0/2, Batch: 300/501, Loss: 0.005382112227380276
Epoch: 0/2, Batch: 350/501, Loss: 0.010400953702628613
Epoch: 0/2, Batch: 400/501, Loss: 0.015848226845264435
Epoch: 0/2, Batch: 450/501, Loss: 0.009098908863961697
Epoch: 0/2, Batch: 500/501, Loss: 0.00862383097410202
Epoch: 1/2, Batch: 0/501, Loss: 0.007520156446844339
Epoch: 1/2, Batch: 50/501, Loss: 0.020851224660873413
Epoch: 1/2, Batch: 100/501, Loss: 0.00462925573810935
Epoch: 1/2, Batch: 150/501, Loss: 0.008697249926626682
Epoch: 1/2, Batch: 200/501, Loss: 0.008650162257254124
Epoch: 1/2, Batch: 250/501, Loss: 0.0043198117054998875
Epoch: 1/2, Batch: 300/501, Loss: 0.009333766996860504
Epoch: 1/

d:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<Figure size 432x288 with 0 Axes>

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.998043,0.998043,0.661738,0.998043,0.483807,0.998043,0.471105,329.770822
1,0.997840,0.997840,0.670685,0.997840,0.489405,0.997840,0.453582,329.905853
2,0.997797,0.997797,0.589022,0.997797,0.476299,0.997797,0.448434,328.510896
3,0.998020,0.998020,0.679552,0.998020,0.479346,0.998020,0.461071,329.259464
4,0.997828,0.997828,0.416478,0.997828,0.482039,0.997828,0.443870,330.423900
5,0.997750,0.997750,0.418082,0.997750,0.481633,0.997750,0.444798,328.952955
6,0.997848,0.997848,0.413388,0.997848,0.479711,0.997848,0.440952,327.921987
7,0.997898,0.997898,0.595042,0.997898,0.512310,0.997898,0.497251,317.634438
8,0.998051,0.998051,0.630952,0.998051,0.555875,0.998051,0.563823,315.369358
9,0.997848,0.997848,0.617625,0.997848,0.482242,0.997848,0.457059,333.940730


In [20]:
results.to_csv(STATS_OUTPUT)